In [1]:
from datasets import load_dataset
import pandas as pd

/home/jaykold/dev/llm-project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
full_data = load_dataset("squad_v2", split="train")

'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /datasets/squad_v2/resolve/main/README.md (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fc1fb557230>: Failed to resolve \'huggingface.co\' ([Errno -3] Temporary failure in name resolution)"))'), '(Request ID: 4f133381-de0c-40c7-b529-ae141cec521b)')' thrown while requesting HEAD https://huggingface.co/datasets/squad_v2/resolve/main/README.md
Retrying in 1s [Retry 1/5].
Generating validation split: 100%|██████████| 11873/11873 [00:00<00:00, 193735.64 examples/s]


In [3]:
shuffled_data = full_data.shuffle(seed=42)

type(shuffled_data)

datasets.arrow_dataset.Dataset

In [4]:
shuffled_data = pd.DataFrame(shuffled_data)

In [5]:
shuffled_data.head()

,id,title,context,question,answers
0,56e0f3907aa994140058e80a,Canon_law,The Roman Catholic Church canon law also inclu...,What term characterizes the intersection of th...,"{'text': ['full union'], 'answer_start': [104]}"
1,571adcf932177014007e9f56,Athanasius_of_Alexandria,Alexandria was the most important trade center...,What was Alexandria known for?,"{'text': ['important trade center'], 'answer_s..."
2,57325b9fe99e3014001e670c,Jehovah%27s_Witnesses,Former members Heather and Gary Botting compar...,How do the leaders of the Jehovah's Witnesses ...,{'text': ['disparaging individual decision-mak...
3,5728d8be4b864d1900164f6b,Estonia,"Historically, the cuisine of Estonia has been ...",What are the most common foods in Estonia?,"{'text': ['black bread, pork, potatoes, and da..."
4,56f6f5e1711bf01900a44898,Classical_music,Many of the instruments used to perform mediev...,What was the medieval flute made from?,"{'text': ['wood'], 'answer_start': [126]}"


In [16]:
#shuffled_data.to_csv("data/squad_v2.csv")

In [85]:
data = shuffled_data.head(600)

In [ ]:
df = pd.DataFrame(data)

In [87]:
df['title']

0                     Canon_law
1      Athanasius_of_Alexandria
2         Jehovah%27s_Witnesses
3                       Estonia
4               Classical_music
                 ...           
595     Northwestern_University
596                       Paper
597                    Zhejiang
598    Brigham_Young_University
599                     YouTube
Name: title, Length: 600, dtype: object

In [88]:
df['title'].nunique()

309

In [89]:
df.sample(5)

,id,title,context,question,answers
2,57325b9fe99e3014001e670c,Jehovah%27s_Witnesses,Former members Heather and Gary Botting compar...,How do the leaders of the Jehovah's Witnesses ...,{'text': ['disparaging individual decision-mak...
275,56e10514e3433e1400422ad8,BeiDou_Navigation_Satellite_System,"In September 2003, China intended to join the ...",Who said in 2008 that China was unsatisfied wi...,{'text': ['the Hong Kong-based South China Mor...
241,56f9519a9e9bad19000a0829,Marshall_Islands,"The Marshall Islands, officially the Republic ...",On what island is the capital of the Marshall ...,"{'text': ['Majuro'], 'answer_start': [692]}"
65,57281d864b864d19001644c6,"St._John%27s,_Newfoundland_and_Labrador","Predominantly Christian, the population of St....",What is the largest religion in St. John's?,"{'text': ['Christian'], 'answer_start': [14]}"
327,5728d4494b864d1900164eff,Karl_Popper,Popper contrasts his views with the notion of ...,Which aspect of organisms changes most radical...,"{'text': ['behaviour'], 'answer_start': [157]}"


In [91]:
df.to_json("data/squad_v2_600.json", orient="records", indent=2)

In [92]:
import json

with open("data/squad_v2_600.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [93]:
data[11]

{'id': '57325c2fe99e3014001e6716',
 'title': 'Jehovah%27s_Witnesses',
 'context': 'Sociologist Rodney Stark states that Jehovah\'s Witness leaders are "not always very democratic" and that members "are expected to conform to rather strict standards," but adds that "enforcement tends to be very informal, sustained by the close bonds of friendship within the group", and that Jehovah\'s Witnesses see themselves as "part of the power structure rather than subject to it." Sociologist Andrew Holden states that most members who join millenarian movements such as Jehovah\'s Witnesses have made an informed choice. However, he also states that defectors "are seldom allowed a dignified exit", and describes the administration as autocratic.',
 'question': "How do Jehovah's Witnesses see themselves in regards to the power structure of the church?",
 'answers': {'text': ['"part of the power structure rather than subject to it."'],
  'answer_start': [330]}}

In [95]:
import json
from urllib.parse import unquote

def clean_text(value):
    """Decode unicode escapes and URL encodings in a string."""
    if not isinstance(value, str):
        return value
    try:
        # Decode URL-encoded characters (%20, %27, etc.)
        value = unquote(value)
        # Decode unicode escapes (\u00f3 etc.)
        #value = value.encode('utf-8').decode('unicode_escape')
        # Decode Python-style backslash escapes (e.g., \' -> ')
        #value = value.encode('utf-8').decode('unicode_escape')
        # Replace underscores with spaces in certain cases
        value = value.replace('_', ' ')
        return value
    except Exception:
        return value  # Fail silently if decoding fails

def clean_data(obj):
    """Recursively clean all strings in a nested structure."""
    if isinstance(obj, dict):
        return {k: clean_data(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [clean_data(item) for item in obj]
    elif isinstance(obj, str):
        return clean_text(obj)
    else:
        return obj


In [96]:
cleaned_data = clean_data(data)

In [102]:
cleaned_data[11]['answers']['text'][0]#.replace('[', "]', "")

'"part of the power structure rather than subject to it."'

In [ ]:
documents = []

for item in data:
    if not item['answers']['text']:
        continue

    documents.append({
        "id": item["id"],
        'title': item['title'],
        "context": item['context'].replace("\n", " "),
        "question": item['question'].replace("\n", " "),
        "answer": item['answers']['text'][0]
    })

IndexError: list index out of range